<a href="https://colab.research.google.com/github/Adlucent/ga4-return-prediction/blob/main/5_Multicollinearity_Check_and_Joining_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Multicollinearity Check and Join Data
At this point, we should check our features for multicollinearity issues.

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = 'adl-analytics' #add proj name
region = "US"  # GCP project region
customer_table_name = 'adl-analytics.return_prediction_ga4.step_4_customer_agg'
transaction_table_name = 'adl-analytics.return_prediction_ga4.step_4_split'

In [ ]:
# If your notebook does not have pandas_gbq you can install it here:
# ! pip install pandas_gbq

In [ ]:
# Google credentials
from google.colab import auth
auth.authenticate_user()

# # BigQuery Magics
# '''BigQuery magics are used to run BigQuery SQL queries in a python environment.
# These queries can also be run in the BigQuery UI '''

# from google.cloud import bigquery
# from google.cloud.bigquery import magics, Client, QueryJobConfig

# magics.context.project = project_name #update project name
# client = bigquery.Client(project=magics.context.project)

# Interface between Jupyter and BigQuery
import pandas_gbq

# data processing libraries
import pandas as pd
import numpy as np

# multicollinearity tools
from statsmodels.stats.outliers_influence import variance_inflation_factor

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# suppress notebook warnings
import warnings
warnings.filterwarnings('ignore')

# dataframe formatting
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# Load customer data

In [ ]:
sql = f"""
SELECT *
FROM `{customer_table_name}`;
"""
customer_data = pandas_gbq.read_gbq(sql, project_id=project_name, location=region, use_bqstorage_api=True)

Downloading: 100%|██████████|


In [ ]:
# create a copy of the imported data to avoid re-importing if we need to revert to original table
df_customer = customer_data.copy()

In [ ]:
df_customer.head()

,user_pseudo_id,ecommerce_transaction_id,nunique_ecommerce_transactions,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,sum_ecommerce_total_item_quantity,sum_total_return_item_quantity,sum_ecommerce_purchase_revenue_in_usd,sum_ecommerce_refund_value_in_usd,sum_ecommerce_tax_value_in_usd,sum_ecommerce_unique_items,max_days_first_session_to_transaction,max_user_ltv_revenue,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_Water

In [ ]:
df_customer.shape

(4466, 226)

In [ ]:
# check for missing values
df_customer.isna().sum().sum()

0

# Customer VIF analysis

In [ ]:
# remove id and split columns from analysis
customer_features = df_customer.iloc[: , 2:-1]
customer_features.head()

,nunique_ecommerce_transactions,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,sum_ecommerce_total_item_quantity,sum_total_return_item_quantity,sum_ecommerce_purchase_revenue_in_usd,sum_ecommerce_refund_value_in_usd,sum_ecommerce_tax_value_in_usd,sum_ecommerce_unique_items,max_days_first_session_to_transaction,max_user_ltv_revenue,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,pre_nunique_event_params_ga_sess

In [ ]:
customer_features = customer_features.astype('float64')

In [ ]:
# first, let's take a look at VIF scores for all original features in the customer dataset
vif_before = pd.DataFrame(
    [variance_inflation_factor(customer_features.values, i) for i in range(customer_features.shape[1])],
    index=customer_features.columns,
    dtype=float,
).reset_index()
vif_before.columns = ('Feature', 'VIF')

In [ ]:
print('Before feature selection:')
vif_before

Before feature selection:


,Feature,VIF
0,nunique_ecommerce_transactions,5053.236
1,avg_event_params_engagement_time_msec,26.310
2,avg_item_price_in_usd,103.844
3,avg_item_promotions,12.451
4,avg_item_quantity,62.263
5,avg_item_refund_in_usd,32.489
6,avg_item_refund_quantity,30.697
7,avg_item_revenue_in_usd,118.084
8,sum_ecommerce_total_item_quantity,579.435
9,sum_total_return_item_quantity,223.094


In [ ]:
vif_before[vif_before['VIF'] > 5].shape[0]

183

Since so many of our features have such high VIF values, let's start by dropping the features with a VIF value over 1000 and recalculate VIF.

In [ ]:
# drop 40 features with VIF over 1000
customer_features = customer_features.drop(list(vif_before[vif_before['VIF'] > 1000].Feature), axis=1)
customer_features.shape

(4466, 183)

In [ ]:
# re-examine VIF values after dropping extreme hight VIF features
vif_trimmed = pd.DataFrame(
    [variance_inflation_factor(customer_features.values, i) for i in range(customer_features.shape[1])],
    index=customer_features.columns,
    dtype=float,
).reset_index()
vif_trimmed.columns = ('Feature', 'VIF')

In [ ]:
print('After trimming extremely high VIF features:')
vif_trimmed

After trimming extremely high VIF features:


,Feature,VIF
0,avg_event_params_engagement_time_msec,16.883
1,avg_item_price_in_usd,86.707
2,avg_item_promotions,10.286
3,avg_item_quantity,45.344
4,avg_item_refund_in_usd,25.289
5,avg_item_refund_quantity,24.457
6,avg_item_revenue_in_usd,98.243
7,sum_ecommerce_total_item_quantity,448.286
8,sum_total_return_item_quantity,199.463
9,sum_ecommerce_purchase_revenue_in_usd,270.271


In [ ]:
vif_trimmed[vif_trimmed['VIF'] > 500].shape[0]

1

After removing extremely high VIF features, only 1 feature still has a VIF value over 500. <br>
Let's start by automatically removing features to get all features to a max VIF value of 10.

In [ ]:
# function to calculate VIF scores and loop through to drop col with max VIF each time until all are less than 10
def vif_and_drop(X, thresh=5.0):
    cols = X.columns
    features = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[features]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[cols[features]].columns[maxloc] + '\' at index: ' + str(maxloc))
            features = np.delete(features, maxloc)
            dropped=True

    X = X[cols[features]]
    return X

In [ ]:
# call the function and update customer_features
customer_features = vif_and_drop(customer_features, thresh=10)

dropping 'sum_item_revenue_in_usd' at index: 28
dropping 'sum_ecommerce_total_item_quantity' at index: 7
dropping 'sum_device_mobile_model_name_Chrome' at index: 33
dropping 'pre_sum_event_name_page_view' at index: 92
dropping 'sum_item_child_category_Other' at index: 79
dropping 'pre_nunique_event_params_ga_sessions' at index: 87
dropping 'sum_event_params_parent_page_PaymentMethod' at index: 55
dropping 'sum_item_parent_category_ShopbyBrand' at index: 73
dropping 'sum_item_refund_quantity' at index: 26
dropping 'sum_ecommerce_purchase_revenue_in_usd' at index: 8
dropping 'pre_sum_device_mobile_brand_name_Apple' at index: 91
dropping 'sum_item_refund_in_usd' at index: 24
dropping 'pre_sum_event_params_parent_page_ShopbyBrand' at index: 128
dropping 'sum_item_parent_category_Stationery' at index: 70
dropping 'sum_event_params_child_page_Kids' at index: 58
dropping 'sum_event_params_parent_page_CheckoutConfirmation' at index: 47
dropping 'pre_sum_device_category_desktop' at index: 85
dr

In [ ]:
# check all final VIF values
vif_reduced = pd.DataFrame(
    [variance_inflation_factor(customer_features.values, i) for i in range(customer_features.shape[1])],
    index=customer_features.columns,
    dtype=float,
).reset_index()
vif_reduced.columns = ('Feature', 'VIF')

In [ ]:
print('All features VIF < 10:')
vif_reduced

All features VIF < 10:


,Feature,VIF
0,avg_event_params_engagement_time_msec,7.136
1,avg_item_price_in_usd,5.159
2,avg_item_promotions,7.040
3,avg_item_refund_in_usd,3.208
4,sum_total_return_item_quantity,5.154
5,sum_ecommerce_refund_value_in_usd,6.960
6,max_days_first_session_to_transaction,1.957
7,max_event_params_engagement_time_msec,3.938
8,max_item_promotions,7.376
9,max_item_quantity,6.165


In [ ]:
customer_features.shape

(4466, 125)

In [ ]:
df_customer_reduced = pd.concat([df_customer.iloc[:, :2], customer_features, df_customer.iloc[:, -1]], axis=1)
df_customer_reduced.head()

,user_pseudo_id,ecommerce_transaction_id,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_refund_in_usd,sum_total_return_item_quantity,sum_ecommerce_refund_value_in_usd,max_days_first_session_to_transaction,max_event_params_engagement_time_msec,max_item_promotions,max_item_quantity,sum_item_promotions,sum_item_quantity,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_Sale,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,pre_sum_event_params_engagement_time_msec,pre_sum_event_name_add_to_cart,pre_sum_event_name_begin_checkout,pre_sum_event_name_select_item,pre_sum_device_category_mobile,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Huawei,pre_sum_device_mobile_brand_name_Microsoft,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_mobile_model_name_iPhone,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Firefox,pre_sum_device_web_info_browser_Safari,pre_sum_geo_country_Canada,pre_sum_geo_country_France,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_New,pre_sum_event_params_parent_page_Other,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_parent_page_ShoppingCart,pre_sum_event_params_child_page_Google,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Kids,pre_sum_event_params_child_page_Notebooks,pre_sum_event_params_child_page_Other,pre_sum_event_params_child_page_Socks,pre_sum_event_params_child_page_Stickers,pre_sum_event_params_child_page_Womens,pre_sum_event_params_child_page_Writing,pre_sum_event_params_child_page_YouTube,pre_sum_item_parent_category_Other,pre_sum_item_parent_category_ShopbyBrand,pre_sum_item_parent_category_Stationery,pre_sum_item_child_category_Bags,pre_sum_item_child_category_Drinkware,pre_sum_item_child_category_MensUnisex,pre_sum_item_child_category_SmallGoods,pre_sum

We've reduced the number of columns from 223 to 125. All scores are now less than 10, so we can move on to the the transactions features.

In [ ]:
# create table schema to prevent errors when uploading data
schema = []

for col, dtype in zip(df_customer_reduced.dtypes.index, df_customer_reduced.dtypes.values):
    col_dict = {}
    col_dict['name'] = col

    if dtype == object:
        typ = 'STRING'
    elif dtype == bool:
        typ = 'BOOLEAN'
    elif dtype == 'datetime64[ns]':
        typ = 'DATETIME'
    elif dtype in (int, 'Int64', 'uint8'):
        typ = 'INTEGER'
    elif dtype == float:
        typ = 'FLOAT'
    else:
        print(dtype)
    col_dict['type'] = typ

    schema.append(col_dict)

In [ ]:
# df_customer_reduced.to_gbq(f'{project_name}.return_prediction_ga4.step_5_customer_reduced',
#                             project_id=project_name,
#                             if_exists='replace',
#                             location=region,
#                             chunksize=100_000,
#                             table_schema=schema)

# Load transaction data

In [ ]:
sql = f"""
SELECT *
FROM `{transaction_table_name}`;
"""
transaction_data = pandas_gbq.read_gbq(sql, project_id=project_name, location=region, use_bqstorage_api=True)

Downloading: 100%|██████████|


In [ ]:
# create a copy of the imported data to avoid re-importing if we need to revert to original table
df_transaction = transaction_data.copy()

In [ ]:
df_transaction.head()

,user_pseudo_id,transaction_date,transaction_ga_session_number,ecommerce_transaction_id,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,total_return_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_refund_value_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_refund_in_usd,sum_item_refund_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_refund_in_usd,max_item_refund_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_refund_in_usd,avg_item_refund_quantity,avg_item_revenue_in_usd,pre_nunique_event_params_ga_sessions,pre_s

In [ ]:
df_transaction.shape

(4466, 225)

In [ ]:
# check for missing values
df_transaction.isna().sum().sum()

0

# Transaction VIF analysis

In [ ]:
# remove features related to refund/return since this is our target
transaction_features = df_transaction.drop([col for col in df_transaction.columns if ('refund' in col) or ('return' in col)], axis=1)
transaction_features.shape

(4466, 217)

In [ ]:
# remove id columns from analysis
transaction_features = transaction_features.drop(['user_pseudo_id', 'transaction_date', 'ecommerce_transaction_id', 'split'], axis=1)
transaction_features.head()

,transaction_ga_session_number,sum_event_params_engagement_time_msec,sum_event_params_session_engaged,user_ltv_revenue,ecommerce_total_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_tax_value_in_usd,ecommerce_unique_items,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_item_revenue_in_usd,sum_event_name_add_to_cart,sum_event_name_begin_checkout,sum_event_name_page_view,sum_event_name_scroll,sum_event_name_select_item,sum_event_name_user_engagement,sum_event_name_view_item,sum_device_category_desktop,sum_device_category_mobile,sum_device_mobile_brand_name_Apple,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Mozilla,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_Chrome,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Edge,sum_device_mobile_model_name_Firefox,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Chrome,sum_device_web_info_browser_Edge,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_geo_country_UnitedStates,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_Apparel,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_CheckoutYourInformation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShopbyBrand,sum_event_params_parent_page_ShoppingCart,sum_event_params_parent_page_Stationery,sum_event_params_child_page_Bags,sum_event_params_child_page_Drinkware,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_MensUnisex,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_SmallGoods,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Apparel,sum_item_parent_category_Collections,sum_item_parent_category_Lifestyle,sum_item_parent_category_New,sum_item_parent_category_Other,sum_item_parent_category_Sale,sum_item_parent_category_ShopbyBrand,sum_item_parent_category_Stationery,sum_item_child_category_Bags,sum_item_child_category_CampusCollection,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_Other,sum_item_child_category_SmallGoods,sum_item_child_category_Womens,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,max_item_quantity,max_item_revenue_in_usd,avg_event_params_engagement_time_msec,avg_item_price_in_usd,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_nunique_event_params_ga_sessions,pre_stdev_max_event_params_engagement_time_msec,pre_stdev_avg_event_params_engagement_time_msec,pre_max_event_params_ga_session_number,pre_max_event_params_engagement_time_msec,pre_avg_event_params_engagement_time_msec,pre_sum_event_params_engagement_time_msec,pr

In [ ]:
transaction_features.shape

(4466, 213)

In [ ]:
transaction_features = transaction_features.astype('float64')

In [ ]:
# first, let's take a look at VIF scores for all original features in the customer dataset
vif_before = pd.DataFrame(
    [variance_inflation_factor(transaction_features.values, i) for i in range(transaction_features.shape[1])],
    index=transaction_features.columns,
    dtype=float,
).reset_index()
vif_before.columns = ('Feature', 'VIF')

In [ ]:
print('Before feature selection:')
vif_before

Before feature selection:


,Feature,VIF
0,transaction_ga_session_number,6.085
1,sum_event_params_engagement_time_msec,18.369
2,sum_event_params_session_engaged,10889.288
3,user_ltv_revenue,5.226
4,ecommerce_total_item_quantity,78.908
5,ecommerce_purchase_revenue_in_usd,60.065
6,ecommerce_tax_value_in_usd,10.440
7,ecommerce_unique_items,17.525
8,sum_item_price_in_usd,44.492
9,sum_item_promotions,2.978


In [ ]:
vif_before[vif_before['VIF'] > 5].shape[0]

167

Again, since so many of our features have such high VIF values, let's start by dropping the features with a VIF value over 500 and recalculate VIF.

In [ ]:
# drop 22 features with VIF over 500
transaction_features = transaction_features.drop(list(vif_before[vif_before['VIF'] > 500].Feature), axis=1)
transaction_features.shape

(4466, 191)

In [ ]:
# re-examine VIF values after dropping extreme hight VIF features
vif_trimmed = pd.DataFrame(
    [variance_inflation_factor(transaction_features.values, i) for i in range(transaction_features.shape[1])],
    index=transaction_features.columns,
    dtype=float,
).reset_index()
vif_trimmed.columns = ('Feature', 'VIF')

In [ ]:
print('After trimming extremely high VIF features:')
vif_trimmed

After trimming extremely high VIF features:


,Feature,VIF
0,transaction_ga_session_number,5.994
1,sum_event_params_engagement_time_msec,17.155
2,user_ltv_revenue,5.175
3,ecommerce_total_item_quantity,78.232
4,ecommerce_purchase_revenue_in_usd,59.434
5,ecommerce_tax_value_in_usd,10.403
6,ecommerce_unique_items,16.900
7,sum_item_price_in_usd,43.652
8,sum_item_promotions,2.942
9,sum_item_quantity,79.052


In [ ]:
vif_trimmed[vif_trimmed['VIF'] > 200].shape[0]

3

After removing extremely high VIF features, only 3 features still has a VIF value over 200. <br>
Let's start by automatically removing features to get all features to a max VIF value of 10.

In [ ]:
# call the function and update customer_features
transaction_features = vif_and_drop(transaction_features, thresh=10)

dropping 'pre_sum_event_name_page_view' at index: 109
dropping 'pre_nunique_event_params_ga_sessions' at index: 101
dropping 'pre_sum_device_mobile_brand_name_Apple' at index: 113
dropping 'pre_sum_device_category_desktop' at index: 111
dropping 'sum_item_child_category_Other' at index: 81
dropping 'pre_sum_event_params_parent_page_ShopbyBrand' at index: 149
dropping 'sum_item_revenue_in_usd' at index: 10
dropping 'sum_event_params_parent_page_CheckoutYourInformation' at index: 46
dropping 'sum_device_category_desktop' at index: 14
dropping 'sum_device_mobile_model_name_Edge' at index: 22
dropping 'pre_sum_item_child_category_Other' at index: 170
dropping 'sum_event_params_parent_page_ShopbyBrand' at index: 51
dropping 'ecommerce_total_item_quantity' at index: 3
dropping 'pre_sum_device_mobile_model_name_Edge' at index: 112
dropping 'pre_sum_device_mobile_model_name_Chrome' at index: 110
dropping 'pre_sum_event_params_child_page_SmallGoods' at index: 151
dropping 'pre_sum_event_params_

In [ ]:
# check all final VIF values
vif_reduced = pd.DataFrame(
    [variance_inflation_factor(transaction_features.values, i) for i in range(transaction_features.shape[1])],
    index=transaction_features.columns,
    dtype=float,
).reset_index()
vif_reduced.columns = ('Feature', 'VIF')

In [ ]:
print('All features VIF < 10:')
vif_reduced

All features VIF < 10:


,Feature,VIF
0,transaction_ga_session_number,5.783
1,user_ltv_revenue,4.541
2,ecommerce_tax_value_in_usd,6.158
3,sum_item_price_in_usd,8.343
4,sum_item_promotions,2.613
5,sum_item_quantity,3.816
6,sum_event_name_begin_checkout,4.143
7,sum_event_name_select_item,2.224
8,sum_device_category_mobile,7.963
9,sum_device_mobile_brand_name_Google,4.632


In [ ]:
transaction_features.shape

(4466, 137)

In [ ]:
df_transaction_reduced = pd.concat([df_transaction.loc[:, ['user_pseudo_id', 'ecommerce_transaction_id']],
                                    transaction_features,
                                    df_transaction.iloc[:, -1]],
                                   axis=1)
df_transaction_reduced.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_stdev_max_event_params_engagement_time_msec,pre_stdev_avg_event_params_engagement_time_msec,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_params_engagement_time_msec,pre_sum_event_name_begin_checkout,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_category_mobile,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Huawei,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_device_web_info_browser_Safari,pre_sum_geo_country_Canada,pre_sum_geo_country_France,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_Other,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_parent_page_ShoppingCart,pre_sum_event_params_child_page_Google,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child

We've reduced the number of columns from 213 to 137. All scores are now less than 10, so we can move on to the the transactions features.

In [ ]:
# create table schema to prevent errors when uploading data
schema = []

for col, dtype in zip(df_transaction_reduced.dtypes.index, df_transaction_reduced.dtypes.values):
    col_dict = {}
    col_dict['name'] = col

    if dtype == object:
        typ = 'STRING'
    elif dtype == bool:
        typ = 'BOOLEAN'
    elif dtype == 'datetime64[ns]':
        typ = 'DATETIME'
    elif dtype in (int, 'Int64', 'uint8'):
        typ = 'INTEGER'
    elif dtype == float:
        typ = 'FLOAT'
    else:
        print(dtype)
    col_dict['type'] = typ

    schema.append(col_dict)

In [ ]:
# df_transaction_reduced.to_gbq(f'{project_name}.return_prediction_ga4.step_5_transaction_reduced',
#                                 project_id=project_name,
#                                 if_exists='replace',
#                                 location=region,
#                                 chunksize=100_000,
#                                 table_schema=schema)

# Join transaction and customer data

In [ ]:
df_customer_reduced.columns = [f'historical_{col}' for col in df_customer_reduced.columns]

In [ ]:
df_merge = pd.merge(df_transaction_reduced.iloc[:, :-1],
                    df_customer_reduced,
                    how='left',
                    left_on=['user_pseudo_id', 'ecommerce_transaction_id'],
                    right_on=['historical_user_pseudo_id', 'historical_ecommerce_transaction_id'])
df_merge.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_stdev_max_event_params_engagement_time_msec,pre_stdev_avg_event_params_engagement_time_msec,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_params_engagement_time_msec,pre_sum_event_name_begin_checkout,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_category_mobile,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Huawei,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_device_web_info_browser_Safari,pre_sum_geo_country_Canada,pre_sum_geo_country_France,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_Other,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_parent_page_ShoppingCart,pre_sum_event_params_child_page_Google,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child

In [ ]:
# drop repeated columns
df_merge = df_merge.drop(['historical_user_pseudo_id', 'historical_ecommerce_transaction_id'], axis=1)
df_merge = df_merge.rename({'historical_split': 'split'}, axis=1)
df_merge.shape

(4466, 265)

# Joint VIF analysis

In [ ]:
# remove id columns from analysis
merged_features = df_merge.drop(['user_pseudo_id', 'ecommerce_transaction_id', 'split'], axis=1)
merged_features.head()

,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_Kids,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_price_in_usd,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_stdev_max_event_params_engagement_time_msec,pre_stdev_avg_event_params_engagement_time_msec,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_params_engagement_time_msec,pre_sum_event_name_begin_checkout,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_category_mobile,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Huawei,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_device_web_info_browser_Safari,pre_sum_geo_country_Canada,pre_sum_geo_country_France,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_Other,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_parent_page_ShoppingCart,pre_sum_event_params_child_page_Google,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Notebooks,pre_sum_event_params_chi

In [ ]:
merged_features.shape

(4466, 262)

In [ ]:
# first, let's take a look at VIF scores for all reduced features across customer and transaction
vif_before = pd.DataFrame(
    [variance_inflation_factor(merged_features.values, i) for i in range(merged_features.shape[1])],
    index=merged_features.columns,
    dtype=float,
).reset_index()
vif_before.columns = ('Feature', 'VIF')

In [ ]:
print('Before feature selection:')
vif_before

Before feature selection:


,Feature,VIF
0,transaction_ga_session_number,7.423
1,user_ltv_revenue,6.918
2,ecommerce_tax_value_in_usd,6.405
3,sum_item_price_in_usd,8.993
4,sum_item_promotions,2.683
5,sum_item_quantity,3.948
6,sum_event_name_begin_checkout,4.500
7,sum_event_name_select_item,2.346
8,sum_device_category_mobile,9.046
9,sum_device_mobile_brand_name_Google,5.025


In [ ]:
vif_before[vif_before['VIF'] > 5].shape[0]

176

Again, since so many of our features have such high VIF values, let's start by dropping the features with a VIF value over 100 and recalculate VIF.

In [ ]:
# drop 48 features with VIF over 100
merged_features = merged_features.drop(list(vif_before[vif_before['VIF'] > 100].Feature), axis=1)
merged_features.shape

(4466, 214)

In [ ]:
# re-examine VIF values after dropping extreme hight VIF features
vif_trimmed = pd.DataFrame(
    [variance_inflation_factor(merged_features.values, i) for i in range(merged_features.shape[1])],
    index=merged_features.columns,
    dtype=float,
).reset_index()
vif_trimmed.columns = ('Feature', 'VIF')

In [ ]:
print('After trimming extremely high VIF features:')
vif_trimmed

After trimming extremely high VIF features:


,Feature,VIF
0,transaction_ga_session_number,7.360
1,user_ltv_revenue,6.537
2,ecommerce_tax_value_in_usd,6.244
3,sum_item_price_in_usd,8.940
4,sum_item_promotions,2.663
5,sum_item_quantity,3.911
6,sum_event_name_begin_checkout,4.327
7,sum_event_name_select_item,2.320
8,sum_device_category_mobile,7.344
9,sum_device_mobile_brand_name_Google,4.909


In [ ]:
vif_trimmed[vif_trimmed['VIF'] > 50].shape[0]

15

After removing extremely high VIF features, 15 features still has a VIF value over 50. <br>
Let's start by automatically removing features to get all features to a max VIF value of 10.

In [ ]:
# call the function and update merged_features
merged_features = vif_and_drop(merged_features, thresh=10)

dropping 'historical_pre_sum_event_params_child_page_Socks' at index: 201
dropping 'historical_pre_sum_device_mobile_model_name_ChromeBook' at index: 178
dropping 'historical_pre_sum_event_params_child_page_Hats' at index: 198
dropping 'historical_pre_sum_geo_country_Other' at index: 184
dropping 'historical_pre_sum_item_child_category_Bags' at index: 201
dropping 'historical_pre_sum_geo_country_India' at index: 183
dropping 'historical_pre_sum_event_params_parent_page_Sale' at index: 195
dropping 'historical_pre_max_event_params_ga_session_number' at index: 203
dropping 'historical_pre_sum_event_params_parent_page_EcoFriendly' at index: 191
dropping 'historical_pre_sum_event_params_parent_page_PaymentMethod' at index: 193
dropping 'historical_pre_sum_traffic_source_medium_referral' at index: 188
dropping 'historical_pre_sum_device_mobile_brand_name_Google' at index: 174
dropping 'historical_pre_sum_device_mobile_model_name_iPad' at index: 177
dropping 'historical_pre_sum_geo_country_C

In [ ]:
# check all final VIF values
vif_reduced = pd.DataFrame(
    [variance_inflation_factor(merged_features.values, i) for i in range(merged_features.shape[1])],
    index=merged_features.columns,
    dtype=float,
).reset_index()
vif_reduced.columns = ('Feature', 'VIF')

In [ ]:
print('All features VIF < 10:')
vif_reduced

All features VIF < 10:


,Feature,VIF
0,transaction_ga_session_number,6.359
1,user_ltv_revenue,6.384
2,ecommerce_tax_value_in_usd,6.141
3,sum_item_price_in_usd,5.886
4,sum_item_promotions,2.644
5,sum_item_quantity,3.881
6,sum_event_name_begin_checkout,4.303
7,sum_event_name_select_item,2.258
8,sum_device_category_mobile,7.194
9,sum_device_mobile_brand_name_Google,4.690


In [ ]:
merged_features.shape

(4466, 181)

In [ ]:
df_merge_reduced = pd.concat([df_merge.loc[:, ['user_pseudo_id', 'ecommerce_transaction_id']],
                              merged_features,
                              df_merge.iloc[:, -1]],
                             axis=1)
df_merge_reduced.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_geo_country_Canada,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Socks,pre_sum_event_params_child_page_Stickers,pre_sum_event_params_child_page_Womens,pre_sum_event_params_child_page_YouTube,pre_sum_item_parent_category_New,pre_sum_item_child_category_Bags,pre_sum_item_child_category_Drinkware,pre_sum_item_child_category_Kids,pre_sum_item_child_category_MensUnisex,pre_sum_item_child_subcategory_Backpacks,pre_sum_item_child_subcategory_ElectronicsAccessories,pre_sum_item_child_subcategory_WaterBottles,days_first_session_to_transaction,his

We've reduced the number of columns from 259 to 181. All scores are now less than 10, so we can move on to the next step. <br>
At this point, we should also re-attach our target column, `ecommerce_refund_value_in_usd`.

In [ ]:
df_merge_final = pd.merge(
    df_merge_reduced,
    df_transaction[['user_pseudo_id', 'ecommerce_transaction_id', 'ecommerce_refund_value_in_usd']],
    how='left',
    on=['user_pseudo_id', 'ecommerce_transaction_id']
)
df_merge_final.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_geo_country_Canada,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Socks,pre_sum_event_params_child_page_Stickers,pre_sum_event_params_child_page_Womens,pre_sum_event_params_child_page_YouTube,pre_sum_item_parent_category_New,pre_sum_item_child_category_Bags,pre_sum_item_child_category_Drinkware,pre_sum_item_child_category_Kids,pre_sum_item_child_category_MensUnisex,pre_sum_item_child_subcategory_Backpacks,pre_sum_item_child_subcategory_ElectronicsAccessories,pre_sum_item_child_subcategory_WaterBottles,days_first_session_to_transaction,his

In [ ]:
# create table schema to prevent errors when uploading data
schema = []

for col, dtype in zip(df_merge_final.dtypes.index, df_merge_final.dtypes.values):
    col_dict = {}
    col_dict['name'] = col

    if dtype == object:
        typ = 'STRING'
    elif dtype == bool:
        typ = 'BOOLEAN'
    elif dtype == 'datetime64[ns]':
        typ = 'DATETIME'
    elif dtype in (int, 'Int64', 'uint8'):
        typ = 'INTEGER'
    elif dtype == float:
        typ = 'FLOAT'
    else:
        print(dtype)
    col_dict['type'] = typ

    schema.append(col_dict)

In [ ]:
# df_merge_final.to_gbq(f'{project_name}.return_prediction_ga4.step_5_merged_final',
#                         project_id=project_name,
#                         if_exists='replace',
#                         location=region,
#                         chunksize=100_000,
#                         table_schema=schema)